# PostgreSQL Schema based GPT Pipeline

Here we shall experiment with a postgreSQL database to experiment with passing Schema information into an LLM from which we can extract SQL Queries as per our need

## Schema Information Extraction Pipeline 

Importing Libraries

In [114]:
import psycopg2 as pg2
import psycopg2.extras
import numpy as np
from numpy import genfromtxt
import pandas as pd

Establishing a connection with a remote database

In [115]:
hostname = "35.225.67.64"
port_id = "5432"
username = "postgres"
pswrd = "Ibrahim19@"
database = "postgres"

In [116]:
conn = pg2.connect(
        host = hostname, 
        dbname = database, 
        user = username,
        password = pswrd,
        port = port_id)


cur = conn.cursor()

This executes the commands to retrieve all the tables inside a Schema 

In [ ]:
# execute the query to retrieve table and column information
cur.execute("""
    SELECT table_name, column_name, data_type 
    FROM information_schema.columns 
    WHERE table_schema = 'quran' 
    ORDER BY table_name, ordinal_position
""")

# fetch all the results as a list of tuples
table_info = cur.fetchall()
# print(table_info)

# close the cursor and connection
# cur.close()
# conn.close()

This stores the Table's name, and all it's columns names and datatypes into a dictionary which refernces the table's name

In [75]:
# create an empty dictionary to store table information
table_dict = {}

# iterate through the results and populate the dictionary
for row in table_info:
    table_name = row[0]
    col_name = row[1]
    col_type = row[2]
    if table_name not in table_dict:
        table_dict[table_name] = []
    table_dict[table_name].append((col_name, col_type))

# convert the dictionary to a NumPy array
table_arr = np.empty((len(table_dict),), dtype=object)
for i, (table_name, cols) in enumerate(table_dict.items()):
    col_arr = np.array(cols, dtype=[('column_name', 'U50'), ('data_type', 'U50')])
    table_arr[i] = (table_name, col_arr)

# create a structured NumPy array with table name and columns
dt = np.dtype([('table_name', 'U50'), ('columns', 'O')])
table_arr = np.array(table_arr, dtype=dt)

# print the resulting NumPy array
# print(table_arr)

This Function converts each Table into an organized list so that passing it to an LLM can b easy

In [95]:
def table_to_list(table):
    table_list = []
    table_list.append(table[0])
    for i in table[1]:
        tempList = [i[0], i[1]]
        table_list.append([i[0], i[1]])
    
    return table_list        

Here we Test out our Function 

In [ ]:
position = table_to_list(table_arr[48])

for i in position:
    print(i)

## Langchain LLM Pipeline

# Extra Code Work

In [130]:
# execute the query to retrieve table and column information
cur.execute("""
SELECT table_name, column_name, data_type 
FROM information_schema.columns
WHERE table_name = 'file';
""")

# fetch all the results as a list of tuples
table_info = cur.fetchall()
print(table_info)

# close the cursor and connection
# cur.close()
# conn.close()

[('file', 'file_id', 'integer'), ('file', 'recitation_id', 'integer'), ('file', 'ayah_key', 'text'), ('file', 'format', 'text'), ('file', 'duration', 'real'), ('file', 'mime_type', 'text'), ('file', 'is_enabled', 'boolean'), ('file', 'url', 'text'), ('file', 'segments_stats', 'json'), ('file', 'segments', 'ARRAY')]


In [4]:
schema_name = 'quran'
table_name = 'qr_posts'   # Enter table name here

query = 'select * from '+schema_name+'.'+table_name

In [29]:
def get_tables():
    cur.execute(""" 
        SELECT table_name, column_name, data_type 
        FROM information_schema.columns 
        WHERE table_schema = 'quran' 
        ORDER BY table_name, ordinal_position
                """)
    tables = cur.fetchall()
    return tables

In [31]:
tables = get_tables()

In [32]:
table_names_arr = np.array(tables)
table_2d = np.reshape(table_names_arr, (-1,1))
print(table_2d)

[['ar_internal_metadata']
 ['key']
 ['character varying']
 ...
 ['words']
 ['text_qpc_nastaleeq_hafs']
 ['character varying']]


In [5]:
cur.execute(query)
result = cur.fetchall()